In [ ]:
import json
import logging
import os
from pathlib import Path
import sys
from seeq import spy
import pandas as pd

"""
This is the context cell that sets up the Data Lab Function (DLF) environment.
Info see: https://support.seeq.com/kb/latest/cloud/data-lab-functions

For this example I'm going to setup python logging to send responses. 
This is also a good tool to use for logging activity in your SDL solution in general. 
"""

def _setup_logging():
    """
    Set up the application for logging
    :return:
    """
    # During initialization set up the logger
    log_path = Path(os.getcwd())
    log_file_name = log_path.joinpath('data_lab_functions.log')
    handler = logging.handlers.RotatingFileHandler(log_file_name,
                                                   maxBytes=1000000,
                                                   backupCount=4)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s -- %(message)s")
    handler.setFormatter(formatter)
    root = logging.getLogger()
    root.setLevel('INFO')
    root.addHandler(handler)

def log_output(context: str = "no-context", message: str = "unknown message", level: str = 'INFO'):
    """
    Output to the log in a structured way
    :param context: Useful to indicate the part of the logic that the message comes from. 
    :param message: Log message content
    :param level: String indicating the log level for the message
    :return:
    """
    log_message = f"context={context}, message={message}"
    if level == 'CRITICAL':
        logging.critical(log_message)
    elif level == 'ERROR':
        logging.error(log_message)
    elif level == 'WARNING':
        logging.warning(log_message)
    elif level == 'INFO':
        logging.info(log_message)
    elif level == 'DEBUG':
        logging.debug(log_message)
    else:
        log_message = f"Log message level NOTSET/unknown.  context={context}, message={message}"
        logging.info(log_message)
        
# Setup the DLF context by intializing the log
_setup_logging()
log_str = f"Ran data_lab_function_demo."
log_output(context="_setup_demo", level='INFO', message=log_str)

# Instantiate the search result Dataframe
global data_result
data_result = pd.DataFrame({'Result': ['Work not yet performed', 'Please run dlf_do_work']})

In [ ]:
# POST /dlf_do_work

# Working function to do work where work is to return an Example data search results dataframe
def dlf_do_work(x, y):
    """
    In this instance the work is to update global data_result dataframe and log something.  This demonstrates doing
    work with the underlying logic and that passed in parameters are received from DLF caller and could be put to work.
    :param x: A demonstration string
    :param y: A demonstration string 
    :return
    """
    try:
        global data_result
        data_result = spy.search({'Name': 'Humid', 'Path': 'Example >> Cooling Tower 1'})
        # log_output(context="METHOD-NOTEBOOK-dlf_do_work", level='INFO', message=f"data_result={data_result}")
        log_output(context="METHOD-NOTEBOOK-dlf_do_work", level='INFO', message=f"Called dlf_do_work: x={x}, y={y}")
    except Exception as e:
        error_msg = f"dlf_do_work encountered an error.  e={e}"
        log_output(context="METHOD-NOTEBOOK-dlf_do_work", level='ERROR', message=error_msg)
        return error_msg
    return "dlf_do_work returned successfully"

# Get the function arguments passed in and put them to work.
args_dict = REQUEST['args']
x = args_dict['x_val']
y = args_dict['y_val']
response = dlf_do_work(x, y)
response = f"dlf_do_work DLF call completed.  Function response={response}"
response

In [ ]:
# GET /get_work_result

# Convert current data_result dataframe to dictionary and return
# Note that if you call this before dlf_do_work the default DataFrame will be returned. 
# If called after dlf_do_work the search results dataframe will be returned. 
data_result_json = data_result.to_json(orient='columns')
response = json.loads(data_result_json)
log_output(context="METHOD-NOTEBOOK-get_work_result", level='INFO', message=f"Called get_work_result.  reponse={response}")
response